# Data Preparation for Media Mix Modeling

This notebook covers essential data preprocessing steps for MMM including:
- Data cleaning and validation
- Feature engineering
- Handling seasonality
- External factor integration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta

# Load sample data (you can replace with your own dataset)
# For now, we'll create synthetic data similar to the previous notebook
np.random.seed(42)
dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='W')
n_weeks = len(dates)

print(f"Working with {n_weeks} weeks of data")

## 1. Data Structure and Validation

In [ ]:
# Create more realistic synthetic data with trends and seasonality
def create_mmm_dataset(n_weeks, start_date='2020-01-01'):
    dates = pd.date_range(start=start_date, periods=n_weeks, freq='W')
    
    # Base trend
    trend = np.linspace(1, 1.5, n_weeks)
    
    # Seasonality (Christmas, summer peaks)
    week_of_year = pd.Series(dates).dt.isocalendar().week
    seasonality = 1 + 0.3 * np.sin(2 * np.pi * week_of_year / 52) + 0.2 * np.sin(4 * np.pi * week_of_year / 52)
    
    # Marketing channels with different patterns
    tv_base = 8000 * trend * seasonality + np.random.normal(0, 1000, n_weeks)
    digital_base = 5000 * trend * (1 + 0.1 * np.sin(2 * np.pi * week_of_year / 52)) + np.random.normal(0, 800, n_weeks)
    radio_base = 3000 * trend + np.random.normal(0, 500, n_weeks)
    print_base = 2000 * trend * (1 - 0.2 * np.sin(2 * np.pi * week_of_year / 52)) + np.random.normal(0, 400, n_weeks)
    
    # Ensure non-negative spending
    tv_spend = np.maximum(tv_base, 0)
    digital_spend = np.maximum(digital_base, 0)
    radio_spend = np.maximum(radio_base, 0)
    print_spend = np.maximum(print_base, 0)
    
    # Conversions influenced by spend (with adstock effects)
    total_spend = tv_spend + digital_spend + radio_spend + print_spend
    base_conversions = 1000 * trend
    media_effect = 0.3 * tv_spend**0.7 + 0.4 * digital_spend**0.8 + 0.2 * radio_spend**0.6 + 0.1 * print_spend**0.5
    conversions = base_conversions + media_effect * seasonality + np.random.normal(0, 200, n_weeks)
    conversions = np.maximum(conversions, 0)
    
    return pd.DataFrame({
        'date': dates,
        'tv_spend': tv_spend,
        'digital_spend': digital_spend,
        'radio_spend': radio_spend,
        'print_spend': print_spend,
        'conversions': conversions,
        'week_of_year': week_of_year,
        'trend': trend,
        'seasonality': seasonality
    })

df = create_mmm_dataset(n_weeks)
print("Dataset created successfully!")
print(df.head())

## 2. Feature Engineering

In [ ]:
# Add time-based features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['is_holiday_season'] = ((df['month'] == 11) | (df['month'] == 12)).astype(int)

# Add lagged variables (adstock effect simulation)
def create_adstock_features(df, channels, max_lag=4, decay_rate=0.7):
    for channel in channels:
        for lag in range(1, max_lag + 1):
            col_name = f'{channel}_lag_{lag}'
            df[col_name] = df[channel].shift(lag) * (decay_rate ** lag)
    return df

channels = ['tv_spend', 'digital_spend', 'radio_spend', 'print_spend']
df = create_adstock_features(df, channels)

# Create total spend metrics
df['total_spend'] = df[channels].sum(axis=1)
df['spend_per_conversion'] = df['total_spend'] / df['conversions']

print(f"Features added. Dataset now has {df.shape[1]} columns")
print("New feature columns:")
print([col for col in df.columns if 'lag' in col or col in ['total_spend', 'spend_per_conversion', 'is_holiday_season']])

## 3. Handling Missing Values and Outliers

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum().sort_values(ascending=False).head(10))

# Handle missing values (forward fill for lagged features)
lag_columns = [col for col in df.columns if 'lag' in col]
df[lag_columns] = df[lag_columns].fillna(0)

# Outlier detection using IQR method
def detect_outliers_iqr(df, columns):
    outliers = {}
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
    return outliers

outliers = detect_outliers_iqr(df, channels + ['conversions'])
print("\nOutliers detected:")
for col, count in outliers.items():
    print(f"{col}: {count} outliers")

## 4. Seasonality and Trend Decomposition

In [ ]:
# Visualize seasonality patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Seasonal Patterns in Marketing Data', fontsize=16)

# Monthly spend patterns
monthly_spend = df.groupby('month')[channels].mean()
axes[0, 0].plot(monthly_spend.index, monthly_spend.values)
axes[0, 0].set_title('Average Monthly Spend by Channel')
axes[0, 0].set_xlabel('Month')
axes[0, 0].set_ylabel('Average Spend')
axes[0, 0].legend(channels)

# Quarterly conversions
quarterly_conv = df.groupby('quarter')['conversions'].mean()
axes[0, 1].bar(quarterly_conv.index, quarterly_conv.values)
axes[0, 1].set_title('Average Quarterly Conversions')
axes[0, 1].set_xlabel('Quarter')
axes[0, 1].set_ylabel('Average Conversions')

# Year-over-year trends
yearly_totals = df.groupby('year')[['total_spend', 'conversions']].sum()
ax1 = axes[1, 0]
ax2 = ax1.twinx()
ax1.bar(yearly_totals.index, yearly_totals['total_spend'], alpha=0.7, color='blue')
ax2.plot(yearly_totals.index, yearly_totals['conversions'], color='red', marker='o')
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Spend', color='blue')
ax2.set_ylabel('Total Conversions', color='red')
axes[1, 0].set_title('Year-over-Year Trends')

# Efficiency over time
monthly_efficiency = df.groupby(['year', 'month'])['spend_per_conversion'].mean().reset_index()
monthly_efficiency['date'] = pd.to_datetime(monthly_efficiency[['year', 'month']].assign(day=1))
axes[1, 1].plot(monthly_efficiency['date'], monthly_efficiency['spend_per_conversion'])
axes[1, 1].set_title('Spend per Conversion Over Time')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Spend per Conversion')

plt.tight_layout()
plt.show()

## 5. Data Scaling and Normalization

In [ ]:
# Prepare features for modeling
feature_columns = channels + lag_columns + ['total_spend', 'is_holiday_season']
feature_columns = [col for col in feature_columns if col in df.columns]

# Remove any remaining NaN values
df_clean = df.dropna(subset=feature_columns + ['conversions'])

# Scale features
scaler = StandardScaler()
df_scaled = df_clean.copy()
df_scaled[feature_columns] = scaler.fit_transform(df_clean[feature_columns])

print(f"Data prepared for modeling:")
print(f"Shape: {df_scaled.shape}")
print(f"Features: {len(feature_columns)}")
print(f"Target variable: conversions")

# Save processed data
df_scaled.to_csv('../data/processed_mmm_data.csv', index=False)
print("\nProcessed data saved to ../data/processed_mmm_data.csv")

## Exercise: Data Quality Assessment

Complete the following data quality checks:

In [ ]:
# TODO: Complete these data quality checks

# 1. Check for data consistency (e.g., no negative spending)
# Your code here

# 2. Validate date ranges and frequency
# Your code here

# 3. Check correlation between channels (multicollinearity)
# Your code here

# 4. Validate business logic (e.g., reasonable spend-to-conversion ratios)
# Your code here

print("Complete the data quality assessment above!")

## Next Steps

In the next module (`03-modeling/`), we'll:
- Build basic MMM models
- Implement adstock and saturation curves
- Use Bayesian approaches for uncertainty quantification

Your data is now ready for modeling!